In [280]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
from scipy.stats import norm, entropy 
from scipy.stats import probplot
import matplotlib.pyplot as plt
from scipy.stats import logistic
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.utils import resample
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from sklearn.preprocessing import Imputer

from pyspark.ml.recommendation import ALS
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.sql.session import SparkSession
import math
from sklearn import preprocessing

In [281]:
column_names=["player","balls"]
data_delivery = pd.read_csv('Delivery_Info.csv')
# df=df.rename(columns={"Unnamed: 0":"Player"})

In [282]:
data_delivery.head(5)

,Match_ID,Innings,Batting_Team,Bowling_Team,Over,Ball,Batsman,Non_Striker,Bowler,Is_Super_Over,...,Bye_Runs,Legbye_Runs,Noball_Runs,Penalty_Runs,Batsman_Runs,Extras,Total_Runs,Player_Dismissed,Dismissal_Type,Fielder
0,1,1,Pune Warriors,Delhi Daredevils,1,1,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Pune Warriors,Delhi Daredevils,1,2,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,2,0,2,NaN,NaN,NaN
2,1,1,Pune Warriors,Delhi Daredevils,1,3,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Pune Warriors,Delhi Daredevils,1,4,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Pune Warriors,Delhi Daredevils,1,5,RV Uthappa,AJ Finch,IK Pathan,0,...,0,0,0,0,1,0,1,NaN,NaN,NaN


In [283]:
le = preprocessing.LabelEncoder()
all_players = set(pd.concat([data_delivery['Batsman'],data_delivery['Non_Striker'],data_delivery['Bowler'],data_delivery['Fielder']], axis = 0))
le.fit(list(all_players))

LabelEncoder()

In [284]:
batsman_bowler_pair = list()
batsman_data = data_delivery.groupby('Batsman')
for batsman_name, value1 in batsman_data:
    bowler_data = value1.groupby('Bowler')
    for bowler_name, value2 in bowler_data:
        batsman_bowler_pair.append([batsman_name, bowler_name, len(value2)])

df = pd.DataFrame.from_records(batsman_bowler_pair, columns = ['Batsman', 'Bowler', 'Balls'])
temp = df.copy(deep = True)
df.Batsman = le.transform(df.Batsman)
df.Bowler = le.transform(df.Bowler)
df.head(5)

,Batsman,Bowler,Balls
0,0,11,9
1,0,24,7
2,0,34,12
3,0,36,3
4,0,73,2


In [124]:
from scipy.sparse import isspmatrix
isspmatrix(df)

False

In [114]:
data_sheet = df.pivot(index='Bowler', columns='Batsman', values='Balls').fillna(0)

print(data_sheet.shape)
data_sheet.head(5)

(404, 514)


Batsman,0,1,2,3,4,5,6,7,8,9,...,612,613,614,615,616,617,618,619,620,621
Bowler,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [170]:
missing_combination = list()
for column in data_sheet.columns:
    for index, row_value in data_sheet[column].iteritems():
        if index != column and row_value == 0:
            missing_combination.append([column, index])

print(len(missing_combination))
missing_combination = pd.DataFrame.from_records(missing_combination, columns = ['Batsman', 'Bowler'])
print(missing_combination.shape)
missing_combination.head(5)


187523
(187523, 2)


,Batsman,Bowler
0,0,1
1,0,3
2,0,4
3,0,5
4,0,6


In [171]:
spark = SparkSession.builder.appName("CF_Bowler_Batsman").getOrCreate()

rdd_df = spark.createDataFrame(df)

In [172]:
als = ALS(maxIter=3,regParam=0.01, userCol="Batsman", itemCol="Bowler", 
          ratingCol="Balls", coldStartStrategy="nan",nonnegative=True)

model= als.fit(rdd_df)

In [173]:
#converting missing combination to rdd Dataframe and then predictions to missing_result
rdd_missing_data = spark.createDataFrame(missing_combination)
predictions = model.transform(rdd_missing_data)
missing_result = predictions.select("*").toPandas()
missing_result.head(5)

,Batsman,Bowler,prediction
0,463,148,0.400305
1,471,148,4.298778
2,496,148,11.981193
3,392,148,1.493243
4,540,148,1.190803


In [156]:
spark.stop()

In [174]:
missing_result = missing_result.sort_values('Batsman')
missing_result.Batsman = le.inverse_transform(missing_result.Batsman)
missing_result.Bowler = le.inverse_transform(missing_result.Bowler)
missing_result.to_csv('avg_balls_played.csv', index = False)
missing_result.head(5)

,Batsman,Bowler,prediction
142583,A Ashish Reddy,Z Khan,5.935884
138639,A Ashish Reddy,K Upadhyay,2.446217
183304,A Ashish Reddy,Rashid Khan,6.391094
176762,A Ashish Reddy,KMDN Kulasekara,7.119550
98178,A Ashish Reddy,SM Boland,3.702372


In [175]:
missing_result.shape

(187523, 3)

In [176]:
df_teamInfo = pd.read_csv('team_players.csv').drop(columns="Unnamed: 0")
df_teamInfo.head(5)

,Chennai Super Kings,Mumbai Indians
0,Abhinav Mukund,Abhishek Nayar
1,Albie Morkel,Ajinkya Rahane
2,Anirudha Srikkanth,André Nel
3,Chamara Kapugedera,Ankeet Chavan
4,Jacob Oram,Ashwell Prince


# Using Surprise Library

In [261]:
sim_options = {
    'name': 'pearson', #similarity measure
    'user_based': False #item based
}

In [262]:
from surprise import SVD, BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import KNNBasic
from surprise import SVDpp



In [263]:
algo = SVD()
reader = Reader(rating_scale=(1, 120))
data = Dataset.load_from_df(df, reader)
trainset_SVD= data.build_full_trainset()
algo.fit(trainset_SVD)

In [264]:
# preparing test set
missing_combination["balls"]=0
missing_combination.head()


,Batsman,Bowler,balls
0,0,1,0
1,0,3,0
2,0,4,0
3,0,5,0
4,0,6,0


In [265]:
test_data=Dataset.load_from_df(missing_combination, reader)
raw_ratings =test_data.raw_ratings
test_data_SVD=test_data.construct_testset(raw_ratings)

In [266]:
predictions = algo.test(test_data_SVD)

In [267]:
Surprise_df=pd.DataFrame(predictions,columns=["Batsman","Bowler","raw","balls","details"]).drop(columns=["raw","details"])
Surprise_df = Surprise_df.sort_values('Batsman')
Surprise_df.Batsman = le.inverse_transform(Surprise_df.Batsman)
Surprise_df.Bowler = le.inverse_transform(Surprise_df.Bowler)
Surprise_df.head(5)

,Batsman,Bowler,balls
0,A Ashish Reddy,A Chandila,4.254747
244,A Ashish Reddy,R Sathish,5.423445
243,A Ashish Reddy,R Rampaul,4.246214
242,A Ashish Reddy,R Parag,6.150376
241,A Ashish Reddy,R Ninan,6.691862


In [274]:
# using SVD
trainset, testset = train_test_split(data, test_size=.25)
algo = SVD(n_factors=110,n_epochs=50)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)


RMSE: 7.5576


7.557643337440573

In [285]:
predictions_SVD=algo.test(test_data_SVD)
predictions_SVD
Surprise_df=pd.DataFrame(predictions_SVD,columns=["Batsman","Bowler","raw","balls","details"]).drop(columns=["raw","details"])
Surprise_df = Surprise_df.sort_values('Batsman')
Surprise_df.Batsman = le.inverse_transform(Surprise_df.Batsman)
Surprise_df.Bowler = le.inverse_transform(Surprise_df.Bowler)
Surprise_df.head(5) # Final missing values using SVD

,Batsman,Bowler,balls
0,A Ashish Reddy,A Chandila,4.599722
244,A Ashish Reddy,R Sathish,6.472763
243,A Ashish Reddy,R Rampaul,4.833072
242,A Ashish Reddy,R Parag,7.100956
241,A Ashish Reddy,R Ninan,6.917495


In [205]:
from surprise import KNNBasic

# Build an algorithm, and train it.
algo = KNNBasic()
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 9.5010


9.500951335076827

In [294]:
bsl_options = {'method': 'als',
               'n_epochs': 7,
               'reg_u': 3,
               'reg_i': 7,
               
               }
algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 8.4235


8.423488425214257

In [292]:
bsl_options = {'method': 'sgd',
               'learning_rate': .005,
               
               }
algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Estimating biases using sgd...
RMSE: 8.4641


8.464125158443679

In [216]:
sim_options = {'name': 'cosine',
               'user_based': False  # compute  similarities between items
               }
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 9.0201


9.02013597953598

In [217]:
sim_options = {'name': 'pearson_baseline',
               'shrinkage': 0  # no shrinkage
               }
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 8.0691


8.069140754026511

In [269]:
algo_svdpp = SVDpp(n_factors=110,n_epochs=50)
algo_svdpp.fit(trainset)
predictions = algo_svdpp.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 7.6177


7.617722120114472

In [277]:
predictions_SVDpp=algo_svdpp.test(test_data_SVD)
Surprise_df=pd.DataFrame(predictions_SVDpp,columns=["Batsman","Bowler","raw","balls","details"]).drop(columns=["raw","details"])
Surprise_df = Surprise_df.sort_values('Batsman')
Surprise_df.Batsman = le.inverse_transform(Surprise_df.Batsman)
Surprise_df.Bowler = le.inverse_transform(Surprise_df.Bowler)
Surprise_df.head(5) #final missing values using SVD_pp

,Batsman,Bowler,balls
0,A Ashish Reddy,A Chandila,6.512449
244,A Ashish Reddy,R Sathish,5.001877
243,A Ashish Reddy,R Rampaul,3.943411
242,A Ashish Reddy,R Parag,6.001619
241,A Ashish Reddy,R Ninan,6.174371


In [279]:
Surprise_df.describe()

,balls
count,187523.000000
mean,6.663835
std,2.074318
min,1.000000
25%,5.452190
50%,6.768665
75%,7.868459
max,42.069177


In [255]:
# Final missing values result DF to CSV file

Surprise_df.to_csv("team_avg_balls_played.csv")

In [257]:
len(Surprise_df.Batsman.unique())

447

In [258]:
len(Surprise_df.Bowler.unique())

376

In [290]:
result_SVD = pd.concat([Surprise_df, temp], axis = 0, sort = True)
result_SVD=result_SVD.drop(columns=["Balls"])
result_SVD.to_csv('team_avg_balls_played_SVD.csv', index = False)